In [5]:
from bs4 import BeautifulSoup as bs
import requests as rq
import pandas as pd
import numpy as np
from tqdm import tqdm
import json

In [6]:
api_url = 'https://www.rbc.ru/v10/ajax/get-news-by-filters'
categories = ['economics', 'sport', 'business', 'technology_and_media', 'finances']

In [7]:
def get_links_and_categories(categories, links_per_category):
    assert links_per_category % 20 == 0, 'error'
    data = []
    for cat in tqdm(categories):
        for offset in np.arange(0, links_per_category, 20):
            r = rq.get(f'{api_url}/?category={cat}&offset={offset}&limit=20')
            soup = bs(r.json()['html'])
            articles = soup.find_all('a', {'class': 'item__link'})
            for el in articles:
                link = el['href']
                data.append({})
                data[-1]['article_id'] = link
                data[-1]['category'] = cat
    return data

In [8]:
data = get_links_and_categories(categories, 1000)
print(len(data))
print(data)

100%|██████████| 5/5 [01:05<00:00, 13.05s/it]

4846
[{'article_id': 'https://www.rbc.ru/economics/16/01/2024/65a63fb89a7947662c322d70', 'category': 'economics'}, {'article_id': 'https://www.rbc.ru/rbcfreenews/65a5540a9a7947976e2d3a5c', 'category': 'economics'}, {'article_id': 'https://www.rbc.ru/economics/16/01/2024/65a52d819a794724e3f9551e', 'category': 'economics'}, {'article_id': 'https://www.rbc.ru/economics/16/01/2024/659e3dc79a7947a15c73d144', 'category': 'economics'}, {'article_id': 'https://www.rbc.ru/rbcfreenews/65a5f58c9a79473ea4303574', 'category': 'economics'}, {'article_id': 'https://www.rbc.ru/rbcfreenews/65a5a6379a7947fc225ee3f0', 'category': 'economics'}, {'article_id': 'https://www.rbc.ru/economics/16/01/2024/65a0fde89a7947399a0590b2', 'category': 'economics'}, {'article_id': 'https://www.rbc.ru/rbcfreenews/65a548c49a794747ab1db23a', 'category': 'economics'}, {'article_id': 'https://www.rbc.ru/economics/15/01/2024/65a4f9309a79479206b9c3c4', 'category': 'economics'}, {'article_id': 'https://www.rbc.ru/economics/15/0

In [9]:
for item in tqdm(data):
    url = item['article_id']
    resp = rq.get(url)
    page = bs(resp.text)
    title = page.find('title').text
    pars = page.find_all('p')
    if pars:
        text = ' '.join(map(lambda p: p.text.replace('<br />','\n').strip(),pars))
    tags = list(map(lambda x: x.text , page.find_all('a', {'class': 'article__tags__item'})))
    # tags = list(map(lambda tag: tag.text, tags))
    item['title'] = title
    item['text'] = text
    item['tags'] = tags


100%|██████████| 4846/4846 [29:21<00:00,  2.75it/s]  


In [10]:
df = pd.DataFrame(data)

In [11]:
df

,article_id,category,title,text,tags
0,https://www.rbc.ru/economics/16/01/2024/65a63f...,economics,В ЦБ заявили о планах привлечь 9 млн россиян к...,К 2030 году в программе долгосрочных сбережени...,"[сбережения, ЦБ, Минфин, НПФ]"
1,https://www.rbc.ru/rbcfreenews/65a5540a9a79479...,economics,Ефимов сообщил детали редевелопмента 10 участк...,В рамках столичной программы комплексного разв...,"[редевелопмент, Москва, Владимир Ефимов]"
2,https://www.rbc.ru/economics/16/01/2024/65a52d...,economics,Число регистраций ИП впервые приблизилось к ми...,По итогам 2023 года число действующих индивиду...,"[ФНС, индивидуальные предприниматели, регистра..."
3,https://www.rbc.ru/economics/16/01/2024/659e3d...,economics,Росстат резко повысил оценку российского ВВП. ...,Росстат в конце прошлого года опубликовал нову...,"[Росстат, ВВП, экономический рост]"
4,https://www.rbc.ru/rbcfreenews/65a5f58c9a79473...,economics,В России проиндексируют более 40 видов выплат ...,"Ряд самых разных выплат, субсидий и пособий пл...","[индексация, выплаты, Минтруд]"
...,...,...,...,...,...
4841,https://www.rbc.ru/finances/24/03/2023/641c185...,finances,Задорнов назвал «хронических больных» среди ба...,Спасение второго банка Швейцарии Credit Suisse...,"[Банки, США, Европа, Credit Suisse]"
4842,https://www.rbc.ru/opinions/finances/24/03/202...,finances,«Нулевой пациент». Чем банковский кризис напом...,Связь между крахом американских банков и пробл...,"[мнение, Михаил Задорнов, Банки]"
4843,https://www.rbc.ru/finances/24/03/2023/641ce28...,finances,"Глава Citi рассказала, как мобильные приложени...",Банковские приложения в смартфонах позволяют к...,"[Citi, Silicon Valley Bank, Банки, мобильные п..."
4844,https://www.rbc.ru/rbcfreenews/641cdf139a79472...,finances,Bloomberg узнал о проверке Credit Suisse насче...,Швейцарские банки Credit Suisse и UBS вошли в ...,"[Credit Suisse, UBS, Швейцария, Банки, санкции..."


In [12]:
data_final = {'catalog': data}
with open('corpus.json', 'w', encoding='utf8') as fout:
     json.dump(data_final, fout, ensure_ascii=False)